In [1]:
import keras 

Using TensorFlow backend.
/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:529: FutureWarn

In [2]:
keras.__version__

'2.3.1'

In [3]:
from __future__ import print_function

from keras.models import Model, load_model 
from keras.layers import Input, LSTM, Dense,TimeDistributed
import numpy as np
import keras.backend as K

In [4]:
def get_dataset(data_path, num_samples):
    input_texts = []
    target_texts = []

    input_characters = set()
    target_characters = set()
    with open(data_path, 'r', encoding='utf-8') as f:
        lines = f.read().split('\n')
    for line in lines[: min(num_samples, len(lines) - 1)]:
        input_text, target_text, _ = line.split('\t')
        # 用tab作用序列的开始，用\n作为序列的结束
        target_text = '\t' + target_text + '\n'

        input_texts.append(input_text)
        target_texts.append(target_text)
        
        for char in input_text:
            if char not in input_characters:
                input_characters.add(char)
        for char in target_text:
            if char not in target_characters:
                target_characters.add(char)
    return input_texts,target_texts,input_characters,target_characters

In [5]:
#------------------------------------------#
#   init初始化部分
#------------------------------------------#
# 每一次输入64个batch
batch_size = 64
# 训练一百个世代
epochs = 100    
# 256维神经元
latent_dim = 256
# 一共10000个样本
num_samples = 120000

In [6]:
# 读取数据集
data_path = './fra.txt'

In [7]:
# 获取数据集
# 其中input_texts为输入的英文字符串
# target_texts为对应的法文字符串

# input_characters用到的所有输入字符,如a,b,c,d,e,……,.,!等
# target_characters用到的所有输出字符
input_texts,target_texts,input_characters,target_characters = get_dataset(data_path, num_samples)

In [8]:
# 对字符进行排序
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
# 计算共用到了什么字符
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
# 计算出最长的序列是多长
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print('一共有多少训练样本：', len(input_texts))
print('多少个英文字母：', num_encoder_tokens)
print('多少个法文字母：', num_decoder_tokens)
print('最大英文序列:', max_encoder_seq_length)
print('最大法文序列:', max_decoder_seq_length)

一共有多少训练样本： 120000
多少个英文字母： 81
多少个法文字母： 108
最大英文序列: 34
最大法文序列: 79


In [9]:
# 建立字母到数字的映射
input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])

In [10]:
#---------------------------------------------------------------------------#

#--------------------------------------#
#   改变数据集的格式
#--------------------------------------#
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

In [11]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    # 为末尾加上" "空格
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    encoder_input_data[i, t + 1:, input_token_index[' ']] = 1.
    # 相当于前一个内容的识别结果，作为输入，传入到解码网络中
    for t, char in enumerate(target_text):
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data不包括第一个tab
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.
    decoder_input_data[i, t + 1:, target_token_index[' ']] = 1.
    decoder_target_data[i, t:, target_token_index[' ']] = 1. 

In [22]:
input_text[:1000]

"You're so beautiful in that dress."

In [12]:
def decode_sequence(input_seq):
    states_value = encoder_model.predict(input_seq)
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    target_seq[0, 0, target_token_index['\t']] = 1.

    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        # 以\t为开头，一个一个向后预测
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char
        # 如果达到结尾
        if (sampled_char == '\n' or len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        states_value = [h, c]
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.


    return decoded_sentence

In [13]:
encoder_inputs = Input(shape=(None, num_encoder_tokens))

encoder_outputs, state_h, state_c = LSTM(latent_dim, return_state=True)(encoder_inputs)

encoder_states = [state_h, state_c]

decoder_inputs = Input(shape=(None, num_decoder_tokens))

decoder_outputs, _, _ = LSTM(latent_dim, return_sequences=True, return_state=True)(decoder_inputs, initial_state=encoder_states)

decoder_outputs = TimeDistributed(Dense(num_decoder_tokens, activation='softmax'))(decoder_outputs)

Instructions for updating:
Colocations handled automatically by placer.


In [ ]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [ ]:
model.summary()

In [ ]:
# 开始训练
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=5,
          validation_split=0.2)
# 保存模型
# model.save('out.h5')

In [14]:
new_model = load_model('./out02.h5') 
model = new_model

Instructions for updating:
Use tf.cast instead.


In [15]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, 81)     0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, None, 108)    0                                            
__________________________________________________________________________________________________
lstm_1 (LSTM)                   [(None, 256), (None, 346112      input_1[0][0]                    
__________________________________________________________________________________________________
lstm_2 (LSTM)                   [(None, None, 256),  373760      input_2[0][0]                    
                                                                 lstm_1[0][1]               

In [16]:
K.clear_session()

encoder_inputs = Input(shape=(None, num_encoder_tokens))

encoder_outputs, state_h, state_c = LSTM(latent_dim, return_state=True)(encoder_inputs)
encoder_states = [state_h, state_c]
encoder_model = Model(encoder_inputs, encoder_states)
encoder_model.load_weights('./out.h5',by_name=True)
encoder_model.summary()

decoder_inputs = Input(shape=(None, num_decoder_tokens))
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))

decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_outputs, state_h, state_c = LSTM(latent_dim, return_sequences=True,
                                        return_state=True)(decoder_inputs, initial_state=decoder_states_inputs)

decoder_states = [state_h, state_c]

decoder_outputs = TimeDistributed(Dense(num_decoder_tokens, activation='softmax'))(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)
decoder_model.load_weights('./out02.h5',by_name=True) 
# 建立序号到字母的映射
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None, 81)          0         
_________________________________________________________________
lstm_1 (LSTM)                [(None, 256), (None, 256) 346112    
Total params: 346,112
Trainable params: 346,112
Non-trainable params: 0
_________________________________________________________________


In [18]:
encoder_input_data[:1]

array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.]]], dtype=float32)

In [30]:
for seq_index in range(100):
    input_seq = np.expand_dims(encoder_input_data[seq_index],axis=0)
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

-
Input sentence: Go.
Decoded sentence: Attendez !

-
Input sentence: Hi.
Decoded sentence: Salut !

-
Input sentence: Hi.
Decoded sentence: Salut !

-
Input sentence: Run!
Decoded sentence: Continuez !

-
Input sentence: Run!
Decoded sentence: Continuez !

-
Input sentence: Who?
Decoded sentence: Qui a l'air ?

-
Input sentence: Wow!
Decoded sentence: Ce soit une bonne nouvelle !

-
Input sentence: Fire!
Decoded sentence: Veuillez vous aider.

-
Input sentence: Help!
Decoded sentence: C'est le printemps.

-
Input sentence: Jump.
Decoded sentence: Souvez-vous !

-
Input sentence: Stop!
Decoded sentence: Assez-le.

-
Input sentence: Stop!
Decoded sentence: Assez-le.

-
Input sentence: Stop!
Decoded sentence: Assez-le.

-
Input sentence: Wait!
Decoded sentence: Regardez-moi.

-
Input sentence: Wait!
Decoded sentence: Regardez-moi.

-
Input sentence: Go on.
Decoded sentence: Souhait-en !

-
Input sentence: Go on.
Decoded sentence: Souhait-en !

-
Input sentence: Go on.
Decoded sentence: S

In [25]:
for x in 'Click the photo to add a tag':
    print(x)

C
l
i
c
k
 
t
h
e
 
p
h
o
t
o
 
t
o
 
a
d
d
 
a
 
t
a
g


In [31]:
test_word = 'Hi.'
enc_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
# 为末尾加上" "空格
for char in test_word:
    enc_input_data[0, t, input_token_index[char]] = 1. 
enc_input_data[0, t + 1:, input_token_index[' ']] = 1. 

In [32]:
enc_input_data[:1]

array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]], dtype=float32)

In [33]:
input_seq = np.expand_dims(enc_input_data[0],axis=0)
decoded_sentence = decode_sequence(input_seq)
print('-')
print('Input sentence:', test_word)
print('Decoded sentence:', decoded_sentence)

-
Input sentence: Hi.
Decoded sentence: La nuit de tous a été déjà fait de moi.

